In [1]:
from config import model_config
from model import VedioRecommender
from dataset import ViewDataSet
import utils

In [2]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from adamp import AdamP

In [3]:
import torch
from torch import nn

In [4]:
from sklearn import metrics
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import json

### 1) Load Model

In [5]:
model = VedioRecommender(model_config)

### 2) Read Data 

In [6]:
df_agg_dataset = utils.open_object("./artifacts/df_agg_dataset.pkl")

In [7]:
df_agg_dataset.head()

,episode_duration,device_first_visit_age,user_age,video_start_hour,video_end_hour,platform_name,user_type,subscription_source,plan_platform,resolution,...,video_streaming_mode,cp_name,product_cat_name,product_lang_name,product_series_cms_id,next_sri_des,hist_sri_des,user_id,sequence_id,label
0,0.372071,0.387207,0.470131,0.623188,0.739130,"[3, 3, 3, 3, 3]","[0, 0, 0, 0, 0]","[2, 2, 2, 2, 2]","[7, 7, 7, 7, 7]","[1, 1, 1, 1, 1]",...,"[2, 2, 2, 2, 2]","[43, 43, 43, 43, 43, 34]","[25, 25, 25, 25, 25, 6]","[2, 2, 2, 2, 2, 2]","[169, 169, 169, 169, 169, 314]",A Jungle Survivor: Tang Ruo Qi (Rebecca Lim) i...,sri_des 1: Mr. Queen: Everyone who lives in a ...,1d0d24b9c6a703cc017342df93df5fee,4,0
1,0.371910,0.387207,0.470131,0.623188,0.666667,"[3, 3, 3, 3, 3]","[0, 0, 0, 0, 0]","[2, 2, 2, 2, 2]","[7, 7, 7, 7, 7]","[1, 1, 1, 1, 1]",...,"[2, 2, 2, 2, 2]","[43, 43, 43, 43, 43, 62]","[25, 25, 25, 25, 25, 4]","[2, 2, 2, 2, 2, 0]","[169, 169, 169, 169, 169, 133]",Sungkyunkwan Scandal: 金允熙为维持家计和赚取弟弟的医药费，迫于无奈女扮...,sri_des 1: Mr. Queen: Everyone who lives in a ...,1d0d24b9c6a703cc017342df93df5fee,7,0
2,0.371910,0.387207,0.470131,0.623188,0.666667,"[3, 3, 3, 3, 3]","[0, 0, 0, 0, 0]","[2, 2, 2, 2, 2]","[7, 7, 7, 7, 7]","[1, 1, 1, 1, 1]",...,"[2, 2, 2, 2, 2]","[43, 43, 43, 43, 43, 35]","[25, 25, 25, 25, 25, 26]","[2, 2, 2, 2, 2, 2]","[169, 169, 169, 169, 169, 707]","Be With You: Before her death, a woman promise...",sri_des 1: Mr. Queen: Everyone who lives in a ...,1d0d24b9c6a703cc017342df93df5fee,10,0
3,0.360397,0.387207,0.470131,0.543478,0.456522,"[3, 3, 3, 3, 3]","[0, 0, 0, 0, 0]","[2, 2, 2, 2, 2]","[7, 7, 7, 7, 7]","[1, 1, 1, 1, 1]",...,"[2, 2, 2, 2, 2]","[43, 43, 43, 43, 36, 36]","[25, 25, 25, 25, 25, 25]","[2, 2, 2, 2, 2, 2]","[169, 169, 169, 169, 984, 984]",Beyond Evil: Police inspectors are often the m...,sri_des 1: Mr. Queen: Everyone who lives in a ...,1d0d24b9c6a703cc017342df93df5fee,18,1
4,0.360397,0.387207,0.470131,0.543478,0.456522,"[3, 3, 3, 3, 3]","[0, 0, 0, 0, 0]","[2, 2, 2, 2, 2]","[7, 7, 7, 7, 7]","[1, 1, 1, 1, 1]",...,"[2, 2, 2, 2, 2]","[43, 43, 43, 43, 36, 28]","[25, 25, 25, 25, 25, 24]","[2, 2, 2, 2, 2, 2]","[169, 169, 169, 169, 984, 243]",Ossan’s Love -in the sky-: [R21 — Mature Theme...,sri_des 1: Mr. Queen: Everyone who lives in a ...,1d0d24b9c6a703cc017342df93df5fee,20,0


In [8]:
sequence_5_features = ['platform_name', 'user_type',
       'subscription_source', 'plan_platform', 'resolution', 'subtitle',
       'screen_mode', 'device_network_mode', 'video_streaming_mode']

sequence_6_features = ['cp_name','product_cat_name', 'product_lang_name', 'product_series_cms_id']

In [9]:
for col in sequence_5_features:
    df_agg_dataset = df_agg_dataset[df_agg_dataset[col].apply(lambda x:len(x)==5)] 

for col in sequence_6_features:
    df_agg_dataset = df_agg_dataset[df_agg_dataset[col].apply(lambda x:len(x)==6)] 

In [10]:
df_train,df_test = train_test_split(df_agg_dataset,test_size=0.3,random_state=33,shuffle = True )

In [11]:
train_dataset = ViewDataSet(df_train)
test_dataset = ViewDataSet(df_test)

In [12]:
batch_size = 32

In [13]:
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=12, shuffle=True)

In [14]:
for inputs in train_loader:
    break

### 3) Training

In [15]:
scores = model(inputs)

In [16]:
BCELoss = nn.BCELoss()

In [17]:
labels = inputs['label']

In [18]:
BCELoss(scores,labels.view(-1,1))

tensor(0.6822, grad_fn=<BinaryCrossEntropyBackward0>)

In [19]:
optimizer = AdamP(model.parameters(),lr=4e-4,
                  betas=(0.9, 0.999), weight_decay=1e-1)

In [20]:
class train_config:
    epoches = 5
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_save_dir = "./artifacts/models"
    train_batch_size = 12
    val_batch_size = int(train_batch_size*1.5)
    eval_steps = (len(train_dataset)//train_batch_size)//3

In [21]:
total_pbar = tqdm(total = len(train_loader)*train_config.epoches,desc = "Model Training",position=0, leave=True)

Model Training:   0%|          | 0/490 [00:00<?, ?it/s]

In [22]:
def evaluate_full_metrics(model,dataset_loader):

    model.eval()

    loss_list = []
    labels_list = []
    pred_list = []
    prob_list = []

    pbar = tqdm(total = len(dataset_loader),desc = "Model Evaluating",position=0, leave=True)


    for inputs in dataset_loader:

        with torch.no_grad():
            
            inputs = utils.to_device(inputs,train_config.device)
            labels = inputs['label'].view(-1,1)
            
            probs = model(inputs)
            
            loss = BCELoss(probs,labels).item()
            loss_list.append(loss)
            
            labels  = labels.detach().cpu().numpy()
            labels_list.extend(labels.flatten())

            probs = probs.detach().cpu().numpy()
            prob_list.extend(probs.flatten())
            pbar.update(1)

    pbar.close()
    
    auc = metrics.roc_auc_score(labels_list, prob_list)
    recall, precision, thres = metrics.precision_recall_curve(labels_list, prob_list)
    
    
    f1 = recall*precision*2 / (recall + precision)
    f1_temp = f1
    f1 = np.nan_to_num(f1,nan = -1)

    arg = f1.argmax()
    
    best_thres = thres[arg]
    best_f1 = f1[arg]
    best_recall = recall[arg]
    best_precision = precision[arg]
    
    pred_list = [1 if prob>=best_thres else 0 for prob in prob_list]
    accuracy = metrics.accuracy_score(labels_list,pred_list)
    
    avg_loss = np.mean(loss_list)
    
    result = {"threshold":best_thres,
              "accuracy":accuracy,
              "recall":best_recall,
              "precision":best_precision,
              "f1":best_f1,'auc':auc,
              'eval_loss':avg_loss} 

    return result,prob_list

In [23]:
result,prob_list = evaluate_full_metrics(model,test_loader)

Model Evaluating: 100%|██████████| 42/42 [01:41<00:00,  2.42s/it]
<ipython-input-22-9613067d0955>:38: RuntimeWarning: invalid value encountered in true_divide
  f1 = recall*precision*2 / (recall + precision)


In [22]:
result

{'threshold': 0.49956766,
 'accuracy': 0.1536926147704591,
 'recall': 0.152,
 'precision': 1.0,
 'f1': 0.2638888888888889,
 'auc': 0.27975232198142413,
 'eval_loss': 0.694588398649579}

In [21]:
def save_model(model, model_save_dir,step,model_metrics):
    model_save_dir = os.path.join(model_save_dir,f"checkpoint-{step}")
    model_name = "pytorch_model.bin"
    train_state_name = "training_state.json"
    os.makedirs(model_save_dir,exist_ok=True)
    
    model_path = os.path.join(model_save_dir,model_name)
    train_state_path = os.path.join(model_save_dir,train_state_name)

    torch.save(model,model_path)
    
    if model_metrics is not None:
        with open(train_state_path,mode = 'w',encoding = 'utf-8-sig') as f:
            model_metrics = {str(k):str(v) for k,v in model_metrics.items()} 
            json.dump(model_metrics,f,indent=4)

In [26]:
len(train_loader)

98

In [28]:
idx

32

In [27]:
idx=0
for inputs in train_loader:
    idx+=1

Model Training:   0%|          | 0/490 [04:37<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [5] at entry 0 and [3] at entry 10

In [22]:
total_pbar = tqdm(total = len(train_loader)*train_config.epoches,desc = "Model Training",position=0, leave=True)

total_batch = 0 
for epoch in range(train_config.epoches):
    print("*"*50 + f"epoch: {epoch + 1}" + "*"*50)
    
    train_losses = []
    
    for inputs in train_loader:
        model = model.train()
        inputs = utils.to_device(inputs,train_config.device)
        labels = inputs['label'].view(-1,1)
        
        
        optimizer.zero_grad()
        
        probs = model(inputs)
        
        loss = BCELoss(probs,labels)
        
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        if (total_batch+1) % train_config.eval_steps ==0:
            model_metrics,_ = evaluate_full_metrics(model,test_loader)
            train_loss = np.mean(train_losses)
            model_metrics['train_loss'] = train_loss
            model_metrics["steps"] = total_batch+1
        
            save_model(model,train_config.model_save_dir,total_batch+1,model_metrics)
            df_metrics_temp = pd.DataFrame([model_metrics])
            display(df_metrics_temp)
            
            model = model.train()
            
    total_batch +=1
    total_pbar.update(1)
        
total_pbar.close()

Model Training:   0%|          | 0/490 [00:00<?, ?it/s]

**************************************************epoch: 1**************************************************


RuntimeError: stack expects each tensor to be equal size, but got [5] at entry 0 and [3] at entry 3